# Singapore Road Data
This is a notebook that will guide you through 
1. Obtaining data from openstreetmaps
2. Generating useful output from the data
  - A visualisation of all the points
  - A text file with the coordinates, road names and number of lanes
  - A dumped dictionary and kd-tree, which can be used to quickly get information regarding a single point


In [1]:
# External imports
import overpy as op
import sys
import concurrent.futures

# Internal imports (classes, functions)
from road_point import RoadPoint
from output_text import export_road_points
from output_scatter import make_gmap
from output_kd_tree import make_kdtree
from speed_limit_scraper import get_speed_limit

  Using cached https://files.pythonhosted.org/packages/9d/20/31f2e2af0dbc05a358259b9cdf67056d34b002d8041944628d05adfab9ba/overpy-0.4.tar.gz
  Running setup.py bdist_wheel for overpy ... done
  Stored in directory: /Users/vigneshshankar/Library/Caches/pip/wheels/10/53/c2/e6b6f97e7bb419193bd3aafbe38628666f0f93a1cec9dc521d
Successfully built overpy


ModuleNotFoundError: No module named 'road_point'